In [1]:
import pandas as pd
import numpy as np

import os
from pathlib import Path
import sys
import glob
import re

import pickle

In [2]:
# Get the directory of the running notebook
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent  # Adjust based on your project structure

sys.path.append(str(project_root))

In [3]:
# konstante Variablen
from Variables.constants import *
from Variables.Classes import * # die Klassen für die Datensätze

In [4]:
from Importfunctions.import_specs import * # allgemeine Datensatzverarbeitungsfunktionen
from Altersgruppierung.age_grouping import *
from Postprocessing.filter import * # Frequenzfilter
from Postprocessing.data_sanitizing import * # Formatierung und Anderes
from Postprocessing.diag_proc import * #häufigste, individuelle Diagnosen
from Farbtonerstellung.hues import *
from Darstellungen.plotting import *
from Darstellungen.axis_config import * # konfiguriere die Achsen je nach Position im Plot

In [5]:
# erhöhe die Seaborn-Auflösung
sns.set_theme(rc={"figure.dpi": 600})

# Lade alle Spektren und Chemshift_Scales als Dataframes in ein Dictionary

In [ ]:
filepath = "Pfad/zu/MAT-Dateien"

In [7]:
spec_dict, chemshift_dict = erstelle_spec_chem_df(filepath)

# Lade alle Befundlisten

In [ ]:
filepath = "Pfad/zu/den/Befundlisten"

In [9]:
# Hole alle Befundlisten in ein Dictionary
df_befund_dict = erstelle_df_befund_dict(filepath)

# Füge die Alterskategorien hinzu

In [10]:
# Füge die Alterskategorien als extra Spalte an
for i, (_,df_befund) in enumerate(df_befund_dict.items()):
    erstelle_alterkateg_df(df_befund)

# Postprocessing Spektren

In [11]:
for i, (k,df_spec) in enumerate (spec_dict.items()):
    print(k)
    # isoliere den realen Teil
    df_spec = real(df_spec)
    # Lowpass
    spec_dict[k] = df_spec.apply(lambda x: lowpass_filter(x, 0.2), axis=1, result_type='expand')

1
2
3
4


# Erstelle die Referenzspektren-Dataframes und entferne sie aus den Gesamtspektren

In [ ]:
# Lade die Referenzspektren
os.chdir("Pfad/zu/den/Referenzspektren")

In [13]:
ref_spec_df_dict = {}
ref_chemshift_df_dict = {}

# Füge die Referenzspektren dem Dataframe zu und lösche diese im Original-Datensatz

In [14]:
for j, (i,df_spec) in enumerate(spec_dict.items()):
    # print(i)
    # print(j)

    df_name = FILE_NAMES[i-1].strip('*') #wegen Zero-Indexing

    df_chemshift = chemshift_dict[i]

    ref_name = f'Referenzspektren_{df_name.split("_")[0]}*' #entferne, die Feldstärke, da diese...
    # irrelevant für die Wahl des Referenzspektrums ist

    ref_df = pd.read_excel(get_fname(ref_name), index_col=0, header=0)
    ref_arr = ref_df.values.flatten()
    ref_arr = ref_arr[~np.isnan(ref_arr)].astype(int)

    match df_name.split("_")[0]:
        case "135ms":
            df_spec_temp = spec_dict[1]
            df_chemshift_temp = chemshift_dict[1]
        case "30ms":
            df_spec_temp = spec_dict[2]
            df_chemshift_temp = chemshift_dict[2]

    ref_spec_df_dict[i] = df_spec_temp.loc[ref_arr]
    ref_chemshift_df_dict[i] = df_chemshift_temp.loc[ref_arr]

    # Füge den Multiindex zur einfacheren Verwendbarkeit ein
    index_temp = ref_df[~pd.isna(ref_df)].stack().index
    ref_spec_df_dict[i].set_index(index_temp,inplace=True)
    ref_chemshift_df_dict[i].set_index(index_temp,inplace=True)

In [15]:
for j, (i,df_spec) in enumerate(spec_dict.items()):
    df_name = FILE_NAMES[i-1].strip('*') #wegen Zero-Indexing
    # Entferne die Spektren aus den ursprünglichen Datensätzen, damit sie nicht zweimal verwendet werden
    # ist im eigenen Loop, weil sonst die Referenzspektren direkt für die anderen Datensätze nicht mehr im ursprünglichen Dataframe enthalten sind
    if df_name=="135ms_1,5T":
        spec_dict[i].drop(ref_arr,inplace=True)
        chemshift_dict[i].drop(ref_arr,inplace=True)
    elif df_name == "30ms_1,5T":
        spec_dict[i].drop(ref_arr,inplace=True)
        chemshift_dict[i].drop(ref_arr,inplace=True)

# Pickle die Datensätze

In [ ]:
os.chdir("path/to/pickled/data")

In [17]:
ds = dataset(df_befund_dict, spec_dict, chemshift_dict, ref_spec_df_dict,ref_chemshift_df_dict)

In [ ]:
# hier soll alles gepicklet werden
with open('df_pickled.pickle','wb') as f:
    pickle.dump(ds,f,pickle.HIGHEST_PROTOCOL)